In [1]:
using CompTime, InteractiveUtils

In [2]:
function mult_vec(v::AbstractVector{T}) where {T}
    l = length(v)
    r = one(T)
    for i in 1:l
        r = r * v[i]
    end
    return r
end

mult_vec (generic function with 1 method)

In [3]:
mult_vec(["a", "b", "c"])

"abc"

In [4]:
@ct_enable function mult_vec_ct(v::AbstractVector{T}, @ct(l)) where {T}
    r = one(T)
    @ct_ctrl for i in 1:l
        r = r * v[@ct(i)]
    end
    return r
end

runtime (generic function with 1 method)

In [5]:
mult_vec_ct(["a", "b", "c"], Val{3})

"abc"

In [6]:
mult_vec_ct(v::AbstractVector{T}) where {T} = mult_vec_ct(v, Val{length(v)})

mult_vec_ct (generic function with 2 methods)

In [7]:
mult_vec_ct(["a", "b", "c"])

"abc"

In [8]:
@code_typed mult_vec_ct([3, 4, 5], Val{3})

CodeInfo(
1 ─ %1 = Base.arrayref(true, v, 1)::Int64
│   %2 = Base.mul_int(1, %1)::Int64
│   %3 = Base.arrayref(true, v, 2)::Int64
│   %4 = Base.mul_int(%2, %3)::Int64
│   %5 = Base.arrayref(true, v, 3)::Int64
│   %6 = Base.mul_int(%4, %5)::Int64
└──      return %6
) => Int64

In [9]:
debug(mult_vec_ct, [3, 4, 5], Val{3}) |> Base.remove_linenums!

quote
    r = one(T)
    begin
        begin
            r = r * v[1]
        end
        begin
            r = r * v[2]
        end
        begin
            r = r * v[3]
        end
    end
    return r
end

In [10]:
@ct_enable function mult_vec_adaptive(v::AbstractVector{T}, @ct(l)) where {T}
    r = one(T)
    @ct_ctrl if l < 5
        @ct_ctrl for i in 1:l
            r = r * v[@ct(i)]
        end
    else
        for i in 1:l
            r = r * v[i]
        end
    end
    return r
end

runtime (generic function with 2 methods)

In [11]:
mult_vec_adaptive(["a", "b", "c", "d","e"], Val{5})

"abcde"

In [12]:
debug(mult_vec_adaptive, ["a", "b", "c", "d","e"], Val{5}) |> Base.remove_linenums!

quote
    r = one(T)
    begin
        for i = 1:l
            r = r * v[i]
        end
    end
    return r
end

In [13]:
mult_vec_adaptive(["a", "b", "c"], Val{3})

"abc"

In [14]:
debug(mult_vec_adaptive, ["a", "b", "c"], Val{3}) |> Base.remove_linenums!

quote
    r = one(T)
    begin
        begin
            begin
                r = r * v[1]
            end
            begin
                r = r * v[2]
            end
            begin
                r = r * v[3]
            end
        end
    end
    return r
end